### Concat all HIBP and Analyze

In [1]:
import glob
import os
import re
import logging
import pandas as pd
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool

In [2]:
# Directory path containing JSON files
directory = 'pwned/'

logging.basicConfig(filename='error_log_read_json.log', level=logging.ERROR,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Get list of JSON file paths in the directory
json_files = glob.glob(directory + '/*.json')

# Define a function for processing a JSON file
def process_json_file(file):
    try:
        df = pd.read_json(file)
        file_name = os.path.basename(file).rstrip('.json')
        df['email'] = file_name
        return df
    except ValueError as e:
        logging.error(f"Error reading file: {file}. Error message: {e}")
        return None

# Create a pool of worker processes
pool = Pool()

# Create a progress bar with tqdm
progress_bar = tqdm(total=len(json_files), desc="Processing JSON files", unit="file")

# Process JSON files in parallel
results = []
for df in pool.imap_unordered(process_json_file, json_files):
    results.append(df)
    progress_bar.update(1)

# Close the pool of worker processes
pool.close()
pool.join()

# Close the progress bar
progress_bar.close()

Processing JSON files: 100%|████████████████████████████████████████████████████████████████████████████████████| 1162292/1162292 [25:25<00:00, 761.83file/s]


In [3]:
# Concatenate the list of DataFrames into a single DataFrame
combined_df = pd.concat(results, ignore_index=True)
del results

In [4]:
breaches = pd.read_json('data/breaches.json')

In [5]:
breached_email_df = pd.merge(combined_df, breaches, on='Name')
breached_email_df.shape

(8643150, 17)

### Let's create an email level dataset for which we have breaches

#### Non-fab
https://haveibeenpwned.com/FAQs#FabricatedBreach

In [6]:
breached_email_df['non_fab'] = ~breached_email_df['IsFabricated'].fillna(1)

#### Serious Dataclasses Breached 
Either data that will help you hack your other accounts, e.g., Mother's Maiden Name, or sensitive personal information

In [7]:
serious_dataclasses = ["Audio recordings",
"Auth tokens", #account cracking
"Bank account numbers",
"Biometric data",
"Browsing histories",
"Chat logs",
"Credit card CVV", #fin
"Credit cards", #fin
#"Credit status information", # don't know what this means
"Drinking habits",
"Driver's licenses",
"Drug habits",
"Email messages",
"Encrypted keys", #account cracking
"Government issued IDs",
#"Health insurance information",
"Historical passwords", #account cracking
"HIV statuses",
#"Mothers maiden names", #account cracking
"Partial credit card data", #fin
"Passport numbers",
"Password hints",
"Passwords", #account cracking
"Personal health data",
"Photos",
"PINs", #account cracking
"Places of birth", #account cracking
"Private messages",
"Security questions and answers", #account cracking
"Sexual fetishes",
"Sexual orientations",
"SMS messages",
"Social security numbers",
"Taxation records"]

In [8]:
breached_email_df['serious'] = breached_email_df['DataClasses'].apply(lambda x: any(string in x for string in serious_dataclasses))

In [ ]:
br_email_df = breached_email_df.assign(count=1).\
    groupby('email').\
    agg(total_breaches = ('count', 'sum'),
    serious_breaches   = ('serious', 'sum'),
    non_fab_breaches   = ('non_fab', 'sum'),
    non_null_uniques   = ('Name', 'nunique')
).reset_index()

### Join to Valid Email List

In [ ]:
valid_emails = pd.read_csv("data/fl_valid_emails.csv")
# can also do less strict is_valid_email
valid_emails = valid_emails[valid_emails['is_valid_email_dns'] == True] 

In [ ]:
# Merge breaches to parsed
valid_emails['email'] = valid_emails['email'].str.lower()
br_email_df['email']  = br_email_df['email'].str.lower()
fin_df = pd.merge(valid_emails, br_email_df, how = 'left', on = 'email')

In [ ]:
fin_df.fillna({'total_breaches': 0, 
               'serious_breaches': 0,
               'non_fab_breaches': 0,
               'non_null_uniques': 0}, inplace = True)

In [ ]:
fin_df[['total_breaches', 'serious_breaches', 'non_fab_breaches', 'non_null_uniques']].describe().round(1)

In [ ]:
fin_df[['total_breaches', 'serious_breaches', 'non_fab_breaches', 'non_null_uniques']].describe().to_markdown()

In [ ]:
(fin_df['total_breaches'] > 0).mean()

### Digital Gap: Sociodemographic Predictors of Breaches

In [ ]:
def calculate_summary_statistics(data_df, groupby_column, value_column, percentiles=None):
    """
    Calculate summary statistics for a given DataFrame.

    Parameters:
        data_df (DataFrame): The DataFrame containing the data.
        groupby_column (str): The column to group by.
        value_column (str): The column for which to calculate statistics.
        percentiles (list, optional): List of percentiles to calculate. Default is None.

    Returns:
        DataFrame: A DataFrame with the summary statistics.
    """
    if percentiles is None:
        percentiles = [25, 50, 75]

    # Calculate summary statistics
    summary_stats = data_df.groupby(groupby_column)[value_column].agg(['count', 'mean', 'std', 'min', 'max']).round(1)

    # Calculate percentile values
    percentile_values = data_df.groupby(groupby_column)[value_column].apply(lambda x: pd.Series(np.nanpercentile(x, percentiles), index=percentiles)).unstack().round(1)

    # Merge summary statistics and percentile values
    summary_with_percentiles = pd.merge(summary_stats, percentile_values, left_index=True, right_index=True)

    # Reorder columns
    desired_columns = ['count', 'mean', 'std', 'min'] + percentiles + ['max']
    summary_with_percentiles = summary_with_percentiles.reindex(columns=desired_columns).reset_index()

    return summary_with_percentiles

In [ ]:
gen_sum = calculate_summary_statistics(fin_df, 
                                   groupby_column='gender', 
                                   value_column='total_breaches',
                                   percentiles = [25, 50, 75])
print(gen_sum)

In [ ]:
gen_sum.to_markdown()

In [ ]:
race_sum = calculate_summary_statistics(fin_df, 
                                   groupby_column='race_lit', 
                                   value_column='total_breaches',
                                   percentiles = [25, 50, 75])
print(race_sum)

In [ ]:
race_sum.to_markdown()

In [ ]:
# Recode Age

fin_df['age'] = 2022 - pd.to_datetime(fin_df['birth_yr'], format='%m/%d/%Y').dt.year
age_buckets = [0, 25, 50, 65, 120]
labels = ['<25', '25--50', '50--65', '65+'] 

fin_df['age_lab'] = pd.cut(fin_df['age'], bins = age_buckets, labels = labels)
fin_df['age_lab'].value_counts()

In [ ]:
print(calculate_summary_statistics(fin_df, 
                                   groupby_column='age_lab', 
                                   value_column='total_breaches',
                                   percentiles = [25, 50, 75]))

In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from matplotlib.ticker import MultipleLocator

lowess = sm.nonparametric.lowess

lowess_model = lowess(fin_df['total_breaches'], fin_df['age'], frac=0.07)

# Create the plot using seaborn and matplotlib
sns.set_style("whitegrid")  # Set the plot style
plt.figure(figsize=(10, 6))
plt.plot(lowess_model[:, 0], lowess_model[:, 1], color='red', linewidth=2, label='Lowess Smoothed')

plt.xlabel('Age', fontsize=14)
plt.ylabel('Total Breaches', fontsize=14)
plt.title('Lowess Smoothed Plot', fontsize=16)
plt.legend(fontsize=12)

# Adjust tick label sizes
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.gca().xaxis.set_major_locator(MultipleLocator(base=10))
plt.xlim(min(fin_df['age']), max(fin_df['age']))

# Add a light gray background to the plot
plt.gca().set_facecolor('#f0f0f0')

# Remove right and top spines
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)


plt.tight_layout()

plt.savefig('figs/age_breaches.png', dpi=300)
plt.show()

In [ ]:
# Let's winsorize or we can lowess over medians
from scipy.stats.mstats import winsorize
winsorized_breaches = winsorize(fin_df['total_breaches'], limits=(0.0, 0.2))

lowess_model = lowess(winsorized_breaches, fin_df['age'], frac=0.07)

# Create the plot using seaborn and matplotlib
sns.set_style("whitegrid")  # Set the plot style
plt.figure(figsize=(10, 6))
plt.plot(lowess_model[:, 0], lowess_model[:, 1], color='red', linewidth=2, label='Lowess Smoothed')

plt.xlabel('Age', fontsize=14)
plt.ylabel('Total Breaches (Winsorized)', fontsize=14)
plt.title('Lowess Smoothed Plot', fontsize=16)
plt.legend(fontsize=12)

# Adjust tick label sizes
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.gca().xaxis.set_major_locator(MultipleLocator(base=10))
plt.xlim(min(fin_df['age']), max(fin_df['age']))

# Add a light gray background to the plot
plt.gca().set_facecolor('#f0f0f0')

# Remove right and top spines
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)


plt.tight_layout()

plt.savefig('figs/age_winsorized_breaches.png', dpi=300)
plt.show()